# Primeira abordagem
### Perfil Digital
Aqui estamos analisando apenas pessoas propensas a possuírem um perfil digital

Principais conclusões:
- Elas priorizam a objetividade
- Pouco contato pelo telefone

In [5]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

query = """SELECT fl_renovou,
NV_FC_CPC___RESPONSAVEL_3M,
NV_BP1_APRESENTACAO_CORRETA_3M,
NV_FC_CONTATO_UTIL_3M,
NV_BP10_ENCERRAMENTO_DA_LIGACAO_3M,
NV_C3_INFORMA_O_MOTIVO_DA_CHAMADA_3M,
NV_D1_EMPATIA_3M,
GC_ISENCAOMORAMULTA_3M,
GC_AUSENCIARENDA_3M,
NV_BP2_INFORMAR_ATRASO_E_VALOR_3M,
PRODUTO_VEICULO_3M,
NV_BP6_GERA_URGENCIA_3M,
NV_C1_PROTOCOLO_DE_SEGURANCA_3M,
MOT_INFORMACAO_3M,
MOT_SERVICOS_3M,
NV_CX_PID_3M,
CX_CORTESIA_3M,
NV_C2_GRAVACAO_DA_LIGACAO_3M,
NV_D2_OBJETIVIDADE_3M,
NV_CX_TITULAR_3M,
NV_FC_PROMESSA_DE_PAGAMENTO_3M,
CX_PESQUISA_SATISFACAO_3M,
CD_CARTAO_3M,
NV_DC3_NAO_E_O_RESPONSAVEL_DO_FINANCIAMENTO_3M,
NV_P1_PAGAMENTO_PARCELA_3M,
NV_CX_LIGAR_DE_VOLTA_3M,
NV_CX_RECHAMADAS_6M,
CX_USABILIDADE_6M,
NV_CX_DIRECIONAMENTO_OUTRO_CANAL_3M,
NV_CX_TRANSFERENCIA_6M,
GC_OFERTANEGOCIACAO_3M,
CD_BOLETO_3M,
P6_RENEGPEDEC_6M,
NV_CD_BOLETO_2DA_VIA_6M,
CX_CLIENTE_NAOCOMPREENDE_3M,
PL_SEM_CONDICOES_3M,
CX_CANCELAMENTO_6M,
CD_SEGURO_6M,
NV_DC10_CLIENTE_ALEGA_PAGAMENTO_3M,
NV_FC_NEGOCIACAO_COM_SUCESSO_3M,
NV_DC1_CLIENTE_NAO_ESTA_COM_O_VALOR_EM_MAOS_6M,
NV_P3_QUITACAO_6M,
MOT_SERVICOS_3M,
CX_BUSCA_INFORMACOES_6M,
NV_BP4_INFORMA_O_DESCONTO_3M,
CD_QUITACAO_6M,
NV_P9_PAGAMENTO_PARCIAL__FLEXIBILIDADE__6M,
NV_CX_ATENDIMENTO_DIFERENCIADO_6M,
CX_OFERTA_6M
FROM `hacka-1.dataset.view_dados_hacka` 
WHERE Sub_Produto in ('CDC VEICULOS DE PASSEIO','PRE APROVADO CDC VEICULOS DE PASSEIO','REFINANCIAMENTO VEICULOS DE PASSEIO')
AND QtParcelas in (24, 36, 48, 54, 60)
AND QtCilindradaVeiculo > 995
AND QtCilindradaVeiculo < 3000
AND QtQuilometragemVeiculo < 150000
AND AaFabricacaoVeiculo >= 2008
AND AaFabricacaoVeiculo <= 2019
AND Capacidade_de_Pagamento_7 in (7,8,9)"""
job = client.query(query)
df = job.to_dataframe()

In [6]:
import pandas as pd

In [7]:
df.shape

(29966, 49)

### Análise do tipo de chamada

In [8]:
df_melt = df.fillna(0).melt(['fl_renovou'], var_name='call',  value_name='vals')

In [9]:
df_renovou = df_melt.loc[df_melt.fl_renovou == 1].groupby(by='call').mean().reset_index().sort_values(by= 'call')

In [10]:
df_geral = df_melt.loc[df_melt.fl_renovou == 0].groupby(by='call').mean().reset_index().sort_values(by= 'call')

In [11]:
df_final = pd.merge(df_geral, df_renovou, on = 'call')

In [12]:
df_final['perc'] = df_final.vals_y/df_final.vals_x
df_final.sort_values(by= 'perc', ascending = False).head()

,call,fl_renovou_x,vals_x,fl_renovou_y,vals_y,perc
34,NV_D2_OBJETIVIDADE_3M,0.0,0.108182,1.0,0.133684,1.235738
28,NV_CX_LIGAR_DE_VOLTA_3M,0.0,0.024194,1.0,0.021053,0.870175
32,NV_CX_TRANSFERENCIA_6M,0.0,0.037359,1.0,0.029474,0.788938
2,CD_QUITACAO_6M,0.0,0.018162,1.0,0.013684,0.753437
10,CX_USABILIDADE_6M,0.0,0.038014,1.0,0.027368,0.719966


Ligações com objetividade são mais frequentes entre os que renovaram o seguro

### Análise de quantidade de chamadas

In [13]:
df_call = df.loc[:,list(df.columns)[1:]]

In [14]:
df_total_calls = pd.concat([df.fl_renovou,df_call.sum(axis = 1)], axis = 1)

Percentual de pessoas que receberam menos de 6 chamadas e não renovaram

In [15]:
len(df_total_calls.loc[df_total_calls.fl_renovou == 0].loc[df_total_calls[0] < 6].index)/len(df_total_calls.loc[df_total_calls.fl_renovou == 0].index)

0.8402950096498484

Percentual de pessoas que renovaram e receberam menos de 6 chamadas

In [16]:
len(df_total_calls.loc[df_total_calls.fl_renovou == 1].loc[df_total_calls[0] < 6].index)/len(df_total_calls.loc[df_total_calls.fl_renovou == 1].index)

0.9021052631578947

In [17]:
import numpy as np
np.histogram(df_total_calls.loc[df_total_calls.fl_renovou == 1][0], bins = 117)

(array([729,  67,  21,  12,  15,  13,   6,  10,   4,   6,   5,   5,   3,
          1,   3,   3,   0,   3,   0,   1,   0,   3,   3,   0,   0,   3,
          2,   2,   0,   2,   3,   0,   0,   1,   1,   1,   0,   1,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   1,   1,   1,
          1,   0,   0,   0,   0,   0,   1,   1,   0,   0,   1,   0,   1,
          1,   0,   0,   1,   0,   1,   0,   1,   1,   0,   1,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          1,   1,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   1,   1,   0,   0,   0,   1]),
 array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
         22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
         33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
         44.,  45.,  46.,  47.,  48.,  49.,  